In [5]:
import sqlite3
import pandas as pd
from soynlp.noun import LRNounExtractor_v2
from collections import defaultdict
import re
import numpy as np
import math
from string import punctuation
from soynlp.word import WordExtractor
from soynlp.utils import DoublespaceLineCorpus

# Stopwords 처리
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation))) # punctuation 제거
pattern2 = re.compile(r'[^가-힣 ]') # 특수문자, 모음, 숫자, 영어 제거
pattern3 = re.compile(r'\s{2,}') # white space 1개로 바꾸기.

class Extracter:
    def __init__(self, df):
        self.df = df
        self.noun_extractor = LRNounExtractor_v2(verbose=True)
        self.word_extractor = WordExtractor(min_frequency=math.floor(len(self.df)*0.0001))
        
    def cleaning(self):
        self.df['head'] = self.df['head'].map(lambda x:pattern3.sub(' ',
                                                        pattern2.sub('',
                                                         pattern1.sub('', x))))
        return self.df

    def extract_nouns(self):
        tempary = np.linspace(0,1,11)
        nouns = [self.noun_extractor.train_extract(_['head'], min_noun_frequency=math.floor(len(self.df)*0.0001)) for _ in [self.df.iloc[math.ceil(len(self.df)*tempary[_]):math.ceil(len(self.df)*tempary[_+1])] for _ in range(len(tempary)-1)]]
        words = {k:v for i in range(len(nouns)) for k,v in nouns[i].items() if len(k) > 1}
        return words

    def search_dict(self,nouns):
        # 사전 검색 결과 없는 단어 추출
        conn = sqlite3.connect('kr_korean.db')
        cur = conn.cursor()
        data = pd.read_sql('SELECT word FROM kr', conn)
        data = ' '.join(data['word'])
        return pd.DataFrame([_ for _ in nouns if _[0] not in data])
    
    # 의미 추출을 위한 training data set 생성
    def extract_sent(self, words):
        sent = defaultdict(lambda:0)
        for w in new_words[0]:
            temp = [s for s in df['head'] if w in s]
            sent[w] = '  '.join(temp)
        return sent
            
    def extract_statistic_value(self, corpus):
#         sent = [s for w in new_words[0] for s in self.df['head'] if w in s]
#         sent = pd.DataFrame(sent)
#         sent.drop_duplicates(keep='first', inplace=True)
        self.word_extractor.train(corpus)
        return self.word_extractor.extract()

In [6]:
conn = sqlite3.connect('Humor.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head',conn)
df.drop_duplicates(keep='first')
ext = Extracter(df)
df = ext.cleaning()

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True))
sent = ext.extract_sent(new_words)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 37878 from 30396 sents. mem=0.386 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=107457, mem=0.409 Gb
[Noun Extractor] batch prediction was completed for 9807 words
[Noun Extractor] checked compounds. discovered 2709 compounds
[Noun Extractor] postprocessing detaching_features : 331 -> 329
[Noun Extractor] postprocessing ignore_features : 329 -> 304
[Noun Extractor] postprocessing ignore_NJ : 304 -> 300
[Noun Extractor] 300 nouns (2709 compounds) with min frequency=30
[Noun Extractor] flushing was done. mem=0.421 Gb                    
[Noun Extractor] 35.41 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 47263 from 30396 sents. mem=0.436 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun 

In [18]:
static = defaultdict(lambda:0)
for k,v in sent.items():
    static[k] = ext.extract_statistic_value([v])[k]

SyntaxError: invalid syntax (<ipython-input-18-fc36b1545b51>, line 2)

In [16]:
static

defaultdict(<function __main__.<lambda>()>,
            {'프듀': Scores(cohesion_forward=0.9871794871794872, cohesion_backward=0.04653371320037987, left_branching_entropy=4.517603982357058, right_branching_entropy=4.418690030596661, left_accessor_variety=155, right_accessor_variety=147, leftside_frequency=2772, rightside_frequency=98),
             '남친': Scores(cohesion_forward=0.9511533242876526, cohesion_backward=0.25704989154013014, left_branching_entropy=4.046033991626005, right_branching_entropy=3.710335838970251, left_accessor_variety=201, right_accessor_variety=211, leftside_frequency=3505, rightside_frequency=474),
             '연습생': Scores(cohesion_forward=0.8862865396071097, cohesion_backward=0.17625643481711992, left_branching_entropy=3.2961960783767164, right_branching_entropy=3.9505852560661285, left_accessor_variety=87, right_accessor_variety=129, leftside_frequency=1333, rightside_frequency=37),
             '강다니엘': Scores(cohesion_forward=0.9959338813940417, cohesion_bac

In [14]:
temp['프듀']

Scores(cohesion_forward=0.9871794871794872, cohesion_backward=0.04653371320037987, left_branching_entropy=4.517603982357058, right_branching_entropy=4.418690030596661, left_accessor_variety=155, right_accessor_variety=147, leftside_frequency=1386, rightside_frequency=49)